In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

def extract_features(data):
    """Extract time and frequency domain features from a given signal."""
    if len(data) == 0:
        return np.zeros(16)  # Return a zero-filled array if data is empty
    
    # Time domain features
    time_domain_features = [
        np.max(data),                            # Max
        np.min(data),                            # Min
        np.mean(data),                           # Mean
        np.std(data),                            # Standard Deviation
        np.mean(np.abs(data - np.mean(data))),   # Mean deviation
        np.sqrt(np.mean(np.square(data))),       # Root mean square
        skew(data),                              # Skewness
        kurtosis(data)                           # Kurtosis
    ]

    # Frequency domain features
    row_fft = np.fft.fft(data)
    row_magnitude = np.abs(row_fft)

    frequency_domain_features = [
        np.sum(row_magnitude * np.arange(len(row_magnitude))) / np.sum(row_magnitude),  # Centroid
        -np.sum(row_magnitude * np.log2(row_magnitude + 1e-10)),                        # Entropy
        np.max(data) - np.min(data),                                                   # Spread
        skew(row_magnitude),                                                           # Skewness
        np.mean(row_magnitude),                                                        # Mean
        kurtosis(row_magnitude),                                                       # Kurtosis
        np.mean(np.abs(np.diff(data))),                                                # Irregularity
        np.var(row_magnitude)                                                          # Variance
    ]
    return time_domain_features + frequency_domain_features

# Load the CSV file into a Pandas DataFrame
file_path = "C:\\Users\\awm21\\Desktop\\Final_Recordings\\Probe1\\CH1\ADC2_CH1\\90.62\_MSPS.csv"
csv2 = pd.read_csv(file_path)

# Assume the first column contains the data
data = csv2.iloc[:, 0].to_numpy()

# Initialize lists to store high (positive) and low (negative) cycles
high_cycles = []
low_cycles = []

current_cycle = []  # To collect values in the current cycle
in_high_cycle = False  # Track whether we're in a high or low cycle

# Separate data into high and low cycles
for value in data:
    if value >= 0:  # High cycle
        if in_high_cycle or not current_cycle:
            current_cycle.append(value)
        else:  # Transition from low to high
            low_cycles.append(current_cycle)
            current_cycle = [value]
        in_high_cycle = True
    else:  # Low cycle
        if not in_high_cycle or not current_cycle:
            current_cycle.append(value)
        else:  # Transition from high to low
            high_cycles.append(current_cycle)
            current_cycle = [value]
        in_high_cycle = False

# Add the last remaining cycle
if current_cycle:
    if in_high_cycle:
        high_cycles.append(current_cycle)
    else:
        low_cycles.append(current_cycle)

# Trim the first and last cycles
high_cycles = high_cycles[1:-1] if len(high_cycles) > 2 else high_cycles
low_cycles = low_cycles[1:-1] if len(low_cycles) > 2 else low_cycles

# Ensure the number of high and low cycles is the same
if len(high_cycles) > len(low_cycles):
    high_cycles = high_cycles[:len(low_cycles)]
elif len(low_cycles) > len(high_cycles):
    low_cycles = low_cycles[:len(high_cycles)]

# Notify user to verify the cycle counts
print(f"Number of high cycles after trimming: {len(high_cycles)}")
print(f"Number of low cycles after trimming: {len(low_cycles)}")

# User prompt for confirmation
input("Please verify the above trimming. Press Enter to confirm and proceed with feature extraction...")

# Convert cycles to float arrays
high_cycles = [np.array(cycle, dtype=float) for cycle in high_cycles]
low_cycles = [np.array(cycle, dtype=float) for cycle in low_cycles]

# Pad the cycles with NaN to make them the same length
max_length = max(max(len(cycle) for cycle in high_cycles), max(len(cycle) for cycle in low_cycles))
high_cycles_padded = [np.pad(cycle, (0, max_length - len(cycle)), constant_values=np.nan) for cycle in high_cycles]
low_cycles_padded = [np.pad(cycle, (0, max_length - len(cycle)), constant_values=np.nan) for cycle in low_cycles]

# Convert to NumPy arrays
high_data = np.array(high_cycles_padded)
low_data = np.array(low_cycles_padded)

# Perform differentiation and integration row-wise
diff_high = np.diff(high_data, axis=1, prepend=high_data[:, 0:1])
diff_low = np.diff(low_data, axis=1, prepend=low_data[:, 0:1])
int_high = np.cumsum(high_data, axis=1)
int_low = np.cumsum(low_data, axis=1)

# Extract features
features_high = [extract_features(cycle[~np.isnan(cycle)]) for cycle in high_data]
features_low = [extract_features(cycle[~np.isnan(cycle)]) for cycle in low_data]
features_diff_high = [extract_features(cycle[~np.isnan(cycle)]) for cycle in diff_high]
features_diff_low = [extract_features(cycle[~np.isnan(cycle)]) for cycle in diff_low]
features_int_high = [extract_features(cycle[~np.isnan(cycle)]) for cycle in int_high]
features_int_low = [extract_features(cycle[~np.isnan(cycle)]) for cycle in int_low]

# Convert features to NumPy arrays
features_high_np = np.array(features_high)
features_low_np = np.array(features_low)
features_diff_high_np = np.array(features_diff_high)
features_diff_low_np = np.array(features_diff_low)
features_int_high_np = np.array(features_int_high)
features_int_low_np = np.array(features_int_low)

# Concatenate all feature arrays
features_dataset = np.concatenate((
    features_high_np,
    features_low_np,
    features_diff_high_np,
    features_diff_low_np,
    features_int_high_np,
    features_int_low_np
), axis=1)

# Specify the save path for the features dataset
save_path = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis\\Probe_3_Features"
os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists

# Save the features dataset
csv_save_path = os.path.join(save_path, "fd_P1_ADC2_Ch1_80.46_MSPS.csv")
npy_save_path = os.path.join(save_path, "fd_P1_ADC2_Ch1_80.46_MSPS.npy")

np.savetxt(csv_save_path, features_dataset, delimiter=",")
np.save(npy_save_path, features_dataset, allow_pickle=True)

# Print summary
print(f"High cycle features shape: {features_high_np.shape}")
print(f"Low cycle features shape: {features_low_np.shape}")
print(f"Diff high cycle features shape: {features_diff_high_np.shape}")
print(f"Diff low cycle features shape: {features_diff_low_np.shape}")
print(f"Int high cycle features shape: {features_int_high_np.shape}")
print(f"Int low cycle features shape: {features_int_low_np.shape}")
print(f"Combined features dataset shape: {features_dataset.shape}")


print(f"Features saved to: {csv_save_path} and {npy_save_path}")


Number of high cycles after trimming: 1418
Number of low cycles after trimming: 1418
High cycle features shape: (1418, 16)
Low cycle features shape: (1418, 16)
Diff high cycle features shape: (1418, 16)
Diff low cycle features shape: (1418, 16)
Int high cycle features shape: (1418, 16)
Int low cycle features shape: (1418, 16)
Combined features dataset shape: (1418, 96)
Features saved to: C:\Users\awm21\Documents\Probe_Vari\Neural_Networks\Features_Vitis\Probe_2_Features\fd_P1_ADC2_Ch1_70.45_MSPS.csv and C:\Users\awm21\Documents\Probe_Vari\Neural_Networks\Features_Vitis\Probe_2_Features\fd_P1_ADC2_Ch1_70.45_MSPS.npy


# NO INTERGAL AND DIFFERENTIAL 


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

def extract_features(data):
    """Extract time and frequency domain features from a given signal."""
    if len(data) == 0:
        return np.zeros(16)  # Return a zero-filled array if data is empty
    
    # Time domain features
    time_domain_features = [
        np.max(data),                            # Max
        np.min(data),                            # Min
        np.mean(data),                           # Mean
        np.std(data),                            # Standard Deviation
        np.mean(np.abs(data - np.mean(data))),   # Mean deviation
        np.sqrt(np.mean(np.square(data))),       # Root mean square
        skew(data),                              # Skewness
        kurtosis(data)                           # Kurtosis
    ]

    # Frequency domain features
    row_fft = np.fft.fft(data)
    row_magnitude = np.abs(row_fft)

    frequency_domain_features = [
        np.sum(row_magnitude * np.arange(len(row_magnitude))) / np.sum(row_magnitude),  # Centroid
        -np.sum(row_magnitude * np.log2(row_magnitude + 1e-10)),                        # Entropy
        np.max(data) - np.min(data),                                                   # Spread
        skew(row_magnitude),                                                           # Skewness
        np.mean(row_magnitude),                                                        # Mean
        kurtosis(row_magnitude),                                                       # Kurtosis
        np.mean(np.abs(np.diff(data))),                                                # Irregularity
        np.var(row_magnitude)                                                          # Variance
    ]
    return time_domain_features + frequency_domain_features

# Load the CSV file into a Pandas DataFrame
file_path = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Datasets\\Probe2\\ADC1_CH1\\15_MSPS.csv"
csv2 = pd.read_csv(file_path)

# Assume the first column contains the data
data = csv2.iloc[:, 0].to_numpy()

# Initialize lists to store high (positive) and low (negative) cycles
high_cycles = []
low_cycles = []

current_cycle = []  # To collect values in the current cycle
in_high_cycle = False  # Track whether we're in a high or low cycle

# Separate data into high and low cycles
for value in data:
    if value >= 0:  # High cycle
        if in_high_cycle or not current_cycle:
            current_cycle.append(value)
        else:  # Transition from low to high
            low_cycles.append(current_cycle)
            current_cycle = [value]
        in_high_cycle = True
    else:  # Low cycle
        if not in_high_cycle or not current_cycle:
            current_cycle.append(value)
        else:  # Transition from high to low
            high_cycles.append(current_cycle)
            current_cycle = [value]
        in_high_cycle = False

# Add the last remaining cycle
if current_cycle:
    if in_high_cycle:
        high_cycles.append(current_cycle)
    else:
        low_cycles.append(current_cycle)

# Trim the first and last cycles
high_cycles = high_cycles[1:-1] if len(high_cycles) > 2 else high_cycles
low_cycles = low_cycles[1:-1] if len(low_cycles) > 2 else low_cycles

# Ensure the number of high and low cycles is the same
if len(high_cycles) > len(low_cycles):
    high_cycles = high_cycles[:len(low_cycles)]
elif len(low_cycles) > len(high_cycles):
    low_cycles = low_cycles[:len(high_cycles)]

# Notify user to verify the cycle counts
print(f"Number of high cycles after trimming: {len(high_cycles)}")
print(f"Number of low cycles after trimming: {len(low_cycles)}")

# User prompt for confirmation
input("Please verify the above trimming. Press Enter to confirm and proceed with feature extraction...")

# Convert cycles to float arrays
high_cycles = [np.array(cycle, dtype=float) for cycle in high_cycles]
low_cycles = [np.array(cycle, dtype=float) for cycle in low_cycles]

# Pad the cycles with NaN to make them the same length
max_length = max(max(len(cycle) for cycle in high_cycles), max(len(cycle) for cycle in low_cycles))
high_cycles_padded = [np.pad(cycle, (0, max_length - len(cycle)), constant_values=np.nan) for cycle in high_cycles]
low_cycles_padded = [np.pad(cycle, (0, max_length - len(cycle)), constant_values=np.nan) for cycle in low_cycles]

# Convert to NumPy arrays
high_data = np.array(high_cycles_padded)
low_data = np.array(low_cycles_padded)

# Extract features
features_high = [extract_features(cycle[~np.isnan(cycle)]) for cycle in high_data]
features_low = [extract_features(cycle[~np.isnan(cycle)]) for cycle in low_data]

# Convert features to NumPy arrays
features_high_np = np.array(features_high)
features_low_np = np.array(features_low)

# Concatenate all feature arrays
features_dataset = np.concatenate((
    features_high_np,
    features_low_np
), axis=1)

# Specify the save path for the features dataset
save_path = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis\\Probe_2_Features"
os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists

# Save the features dataset
csv_save_path = os.path.join(save_path, "fd_P2_ADC1_Ch1_15_MSPS.csv")
npy_save_path = os.path.join(save_path, "fd_P2_ADC1_Ch1_15_MSPS.npy")

np.savetxt(csv_save_path, features_dataset, delimiter=",")
np.save(npy_save_path, features_dataset, allow_pickle=True)

# Print summary
print(f"High cycle features shape: {features_high_np.shape}")
print(f"Low cycle features shape: {features_low_np.shape}")
print(f"Combined features dataset shape: {features_dataset.shape}")

print(f"Features saved to: {csv_save_path} and {npy_save_path}")

Number of high cycles after trimming: 25
Number of low cycles after trimming: 25


# New changes

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from scipy.fft import fft

def extract_features(data):
    """Extract time and frequency domain features from a given signal."""
    if len(data) == 0:
        return np.zeros(16)  # Return a zero-filled array if data is empty
    
    # Time domain features
    time_domain_features = [
        np.max(data),                            # Max
        np.min(data),                            # Min
        np.mean(data),                           # Mean
        np.std(data),                            # Standard Deviation
        np.mean(np.abs(data - np.mean(data))),   # Mean deviation
        np.sqrt(np.mean(np.square(data))),       # Root mean square
        skew(data),                              # Skewness
        kurtosis(data),                          # Kurtosis
        np.max(data) - np.min(data),             # Peak-to-peak amplitude
        len(np.where(np.diff(np.sign(data)))[0]) # Zero-crossing rate
    ]

    # Frequency domain features
    row_fft = np.fft.fft(data)
    row_magnitude = np.abs(row_fft)

    frequency_domain_features = [
        np.sum(row_magnitude * np.arange(len(row_magnitude))) / np.sum(row_magnitude),  # Centroid
        -np.sum(row_magnitude * np.log2(row_magnitude + 1e-10)),                        # Entropy
        np.max(data) - np.min(data),                                                   # Spread
        skew(row_magnitude),                                                           # Skewness
        np.mean(row_magnitude),                                                        # Mean
        kurtosis(row_magnitude),                                                       # Kurtosis
        np.mean(np.abs(np.diff(data))),                                                # Irregularity
        np.var(row_magnitude),                                                         # Variance
        np.argmax(row_magnitude)                                                       # Dominant frequency
    ]
    return time_domain_features + frequency_domain_features

def plot_cycles(cycles, title):
    """Plot the first few cycles for visualization."""
    plt.figure(figsize=(12, 6))
    for i in range(min(5, len(cycles))):  # Plot first 5 cycles
        plt.plot(cycles[i], label=f'Cycle {i+1}')
    plt.title(title)
    plt.xlabel("Sample Index")
    plt.ylabel("Magnitude")
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_feature_histograms(features, title):
    """Plot histograms of the extracted features."""
    plt.figure(figsize=(12, 8))
    for i in range(features.shape[1]):
        plt.subplot(4, 4, i+1)
        plt.hist(features[:, i], bins=20, edgecolor='black')
        plt.title(f'Feature {i+1}')
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# Load the CSV file into a Pandas DataFrame
file_path = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Datasets\\Probe2\\ADC1_CH1\\15_MSPS.csv"
try:
    csv_data = pd.read_csv(file_path)
    print("CSV file loaded successfully.")
except Exception as e:
    print(f"Error loading CSV file: {e}")
    exit()

# Assume the first column contains the data
data = csv_data.iloc[:, 0].to_numpy()

# Initialize lists to store high (positive) and low (negative) cycles
high_cycles = []
low_cycles = []

current_cycle = []  # To collect values in the current cycle
in_high_cycle = False  # Track whether we're in a high or low cycle

# Separate data into high and low cycles
for value in data:
    if value >= 0:  # High cycle
        if in_high_cycle or not current_cycle:
            current_cycle.append(value)
        else:  # Transition from low to high
            low_cycles.append(current_cycle)
            current_cycle = [value]
        in_high_cycle = True
    else:  # Low cycle
        if not in_high_cycle or not current_cycle:
            current_cycle.append(value)
        else:  # Transition from high to low
            high_cycles.append(current_cycle)
            current_cycle = [value]
        in_high_cycle = False

# Add the last remaining cycle
if current_cycle:
    if in_high_cycle:
        high_cycles.append(current_cycle)
    else:
        low_cycles.append(current_cycle)

# Trim the first and last cycles
high_cycles = high_cycles[1:-1] if len(high_cycles) > 2 else high_cycles
low_cycles = low_cycles[1:-1] if len(low_cycles) > 2 else low_cycles

# Ensure the number of high and low cycles is the same
if len(high_cycles) > len(low_cycles):
    high_cycles = high_cycles[:len(low_cycles)]
elif len(low_cycles) > len(high_cycles):
    low_cycles = low_cycles[:len(high_cycles)]

print(f"Number of high cycles after trimming: {len(high_cycles)}")
print(f"Number of low cycles after trimming: {len(low_cycles)}")

# Plot the first few high and low cycles
plot_cycles(high_cycles, "First Few Trimmed High PWM Cycles")
plot_cycles(low_cycles, "First Few Trimmed Low PWM Cycles")

# Convert cycles to float arrays
high_cycles = [np.array(cycle, dtype=float) for cycle in high_cycles]
low_cycles = [np.array(cycle, dtype=float) for cycle in low_cycles]

# Pad the cycles with NaN to make them the same length
max_length = max(max(len(cycle) for cycle in high_cycles), max(len(cycle) for cycle in low_cycles))
high_cycles_padded = [np.pad(cycle, (0, max_length - len(cycle)), constant_values=np.nan) for cycle in high_cycles]
low_cycles_padded = [np.pad(cycle, (0, max_length - len(cycle)), constant_values=np.nan) for cycle in low_cycles]

# Convert to NumPy arrays
high_data = np.array(high_cycles_padded)
low_data = np.array(low_cycles_padded)

# Extract features
features_high = [extract_features(cycle[~np.isnan(cycle)]) for cycle in high_data]
features_low = [extract_features(cycle[~np.isnan(cycle)]) for cycle in low_data]

# Convert features to NumPy arrays
features_high_np = np.array(features_high)
features_low_np = np.array(features_low)

# Concatenate all feature arrays
features_dataset = np.concatenate((features_high_np, features_low_np), axis=0)

# Plot histograms of the extracted features
plot_feature_histograms(features_high_np, "High Cycle Features")
plot_feature_histograms(features_low_np, "Low Cycle Features")

# Specify the save path for the features dataset
save_path = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis\\Probe_2_Features"
os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists

# Save the features dataset
csv_save_path = os.path.join(save_path, "fd_P2_ADC1_Ch1_15_MSPS.csv")
npy_save_path = os.path.join(save_path, "fd_P2_ADC1_Ch1_15_MSPS.npy")

np.savetxt(csv_save_path, features_dataset, delimiter=",")
np.save(npy_save_path, features_dataset, allow_pickle=True)

# Print summary
print(f"High cycle features shape: {features_high_np.shape}")
print(f"Low cycle features shape: {features_low_np.shape}")
print(f"Combined features dataset shape: {features_dataset.shape}")
print(f"Features saved to: {csv_save_path} and {npy_save_path}")

# Automated Feature Extraction

In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from scipy.fft import fft

# Define the base directory where the datasets are stored
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Datasets"

# Define the sampling rates to process
sampling_rates = [
    "10_MSPS.csv", "20_MSPS.csv", "30_MSPS.csv", "40.28_MSPS.csv",
    "50_MSPS.csv", "60.42_MSPS.csv", "70.45_MSPS.csv", "80.56_MSPS.csv",
    "90.62_MSPS.csv", "100_MSPS.csv", "125_MSPS.csv", "150_MSPS.csv",
    "200_MSPS.csv", "250_MSPS.csv", "300_MSPS.csv"
]

# Define the output directory where features will be saved
output_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

# Create subfolders for Probe1, Probe2, and Probe3
for probe in ["P1_fd", "P2_fd", "P3_fd"]:
    os.makedirs(os.path.join(output_directory, probe), exist_ok=True)

def extract_features(data):
    """Extract time and frequency domain features from a given signal."""
    if len(data) == 0:
        return np.zeros(16)  # Return a zero-filled array if data is empty
    
    # Time domain features
    time_domain_features = [
        np.max(data),                            # Max
        np.min(data),                            # Min
        np.mean(data),                           # Mean
        np.std(data),                            # Standard Deviation
        np.mean(np.abs(data - np.mean(data))),   # Mean deviation
        np.sqrt(np.mean(np.square(data))),       # Root mean square
        skew(data),                              # Skewness
        kurtosis(data),                          # Kurtosis
        np.max(data) - np.min(data),             # Peak-to-peak amplitude
        len(np.where(np.diff(np.sign(data)))[0]) # Zero-crossing rate
    ]

    # Frequency domain features
    row_fft = np.fft.fft(data)
    row_magnitude = np.abs(row_fft)

    frequency_domain_features = [
        np.sum(row_magnitude * np.arange(len(row_magnitude))) / np.sum(row_magnitude),  # Centroid
        -np.sum(row_magnitude * np.log2(row_magnitude + 1e-10)),                        # Entropy
        np.max(data) - np.min(data),                                                   # Spread
        skew(row_magnitude),                                                           # Skewness
        np.mean(row_magnitude),                                                        # Mean
        kurtosis(row_magnitude),                                                       # Kurtosis
        np.mean(np.abs(np.diff(data))),                                                # Irregularity
        np.var(row_magnitude),                                                         # Variance
        np.argmax(row_magnitude)                                                       # Dominant frequency
    ]
    return time_domain_features + frequency_domain_features

def process_file(file_path, output_path):
    """Process a single file and save the extracted features."""
    try:
        # Load the CSV file
        csv_data = pd.read_csv(file_path)
        print(f"Processing file: {file_path}")
        
        # Assume the first column contains the data
        data = csv_data.iloc[:, 0].to_numpy()

        # Initialize lists to store high (positive) and low (negative) cycles
        high_cycles = []
        low_cycles = []

        current_cycle = []  # To collect values in the current cycle
        in_high_cycle = False  # Track whether we're in a high or low cycle

        # Separate data into high and low cycles
        for value in data:
            if value >= 0:  # High cycle
                if in_high_cycle or not current_cycle:
                    current_cycle.append(value)
                else:  # Transition from low to high
                    low_cycles.append(current_cycle)
                    current_cycle = [value]
                in_high_cycle = True
            else:  # Low cycle
                if not in_high_cycle or not current_cycle:
                    current_cycle.append(value)
                else:  # Transition from high to low
                    high_cycles.append(current_cycle)
                    current_cycle = [value]
                in_high_cycle = False

        # Add the last remaining cycle
        if current_cycle:
            if in_high_cycle:
                high_cycles.append(current_cycle)
            else:
                low_cycles.append(current_cycle)

        # Trim the first and last cycles
        high_cycles = high_cycles[1:-1] if len(high_cycles) > 2 else high_cycles
        low_cycles = low_cycles[1:-1] if len(low_cycles) > 2 else low_cycles

        # Ensure the number of high and low cycles is the same
        if len(high_cycles) > len(low_cycles):
            high_cycles = high_cycles[:len(low_cycles)]
        elif len(low_cycles) > len(high_cycles):
            low_cycles = low_cycles[:len(high_cycles)]

        # Print the number of high and low cycles after trimming
        print(f"Number of high cycles after trimming: {len(high_cycles)}")
        print(f"Number of low cycles after trimming: {len(low_cycles)}")

        # Convert cycles to float arrays
        high_cycles = [np.array(cycle, dtype=float) for cycle in high_cycles]
        low_cycles = [np.array(cycle, dtype=float) for cycle in low_cycles]

        # Pad the cycles with NaN to make them the same length
        max_length = max(max(len(cycle) for cycle in high_cycles), max(len(cycle) for cycle in low_cycles))
        high_cycles_padded = [np.pad(cycle, (0, max_length - len(cycle)), constant_values=np.nan) for cycle in high_cycles]
        low_cycles_padded = [np.pad(cycle, (0, max_length - len(cycle)), constant_values=np.nan) for cycle in low_cycles]

        # Convert to NumPy arrays
        high_data = np.array(high_cycles_padded)
        low_data = np.array(low_cycles_padded)

        # Extract features
        features_high = [extract_features(cycle[~np.isnan(cycle)]) for cycle in high_data]
        features_low = [extract_features(cycle[~np.isnan(cycle)]) for cycle in low_data]

        # Convert features to NumPy arrays
        features_high_np = np.array(features_high)
        features_low_np = np.array(features_low)

        # Print the shape of high and low cycle features
        print(f"High cycle features shape: {features_high_np.shape}")
        print(f"Low cycle features shape: {features_low_np.shape}")

        # Concatenate all feature arrays
        features_dataset = np.concatenate((features_high_np, features_low_np), axis=1)

        # Print the shape of the combined features dataset
        print(f"Combined features dataset shape: {features_dataset.shape}")

        # Save the features dataset
        np.savetxt(output_path, features_dataset, delimiter=",")
        np.save(output_path.replace(".csv", ".npy"), features_dataset, allow_pickle=True)

        print(f"Features saved to: {output_path}")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        
# Iterate over all probes and channels
for probe in ["Probe1", "Probe2", "Probe3"]:
    for channel in ["ADC1_CH1", "ADC1_CH2", "ADC2_CH1", "ADC2_CH2", "ADC3_CH1", "ADC3_CH2"]:
        for rate in sampling_rates:
            file_path = os.path.join(base_directory, probe, channel, rate)
            
            # Split the rate into filename and extension
            filename, ext = os.path.splitext(rate)
            
            # Replace dots in the filename (not in the extension)
            filename = filename.replace('.', '_')
            
            # Reconstruct the output path with the correct .csv extension
            output_path = os.path.join(output_directory, f"P{probe[-1]}_fd", f"fd_{probe}_{channel}_{filename}{ext}")
            
            if os.path.exists(file_path):
                # Process the file and extract features
                process_file(file_path, output_path)
                
                # Load the saved features to verify the shape
                saved_features = np.load(output_path.replace(".csv", ".npy"))
                print(f"Shape of features for {probe}, {channel}, {rate}: {saved_features.shape}")
            else:
                print(f"File not found: {file_path}")

Processing file: C:\Users\awm21\Documents\Probe_Vari\Datasets\Probe1\ADC1_CH1\10_MSPS.csv
Number of high cycles after trimming: 9998
Number of low cycles after trimming: 9998
High cycle features shape: (9998, 19)
Low cycle features shape: (9998, 19)
Combined features dataset shape: (9998, 38)
Features saved to: C:\Users\awm21\Documents\Probe_Vari\Neural_Networks\Features_Vitis\P1_fd\fd_Probe1_ADC1_CH1_10_MSPS.csv
Shape of features for Probe1, ADC1_CH1, 10_MSPS.csv: (9998, 38)
Processing file: C:\Users\awm21\Documents\Probe_Vari\Datasets\Probe1\ADC1_CH1\20_MSPS.csv
Number of high cycles after trimming: 4998
Number of low cycles after trimming: 4998
High cycle features shape: (4998, 19)
Low cycle features shape: (4998, 19)
Combined features dataset shape: (4998, 38)
Features saved to: C:\Users\awm21\Documents\Probe_Vari\Neural_Networks\Features_Vitis\P1_fd\fd_Probe1_ADC1_CH1_20_MSPS.csv
Shape of features for Probe1, ADC1_CH1, 20_MSPS.csv: (4998, 38)
Processing file: C:\Users\awm21\Docume